<img src="https://raw.githubusercontent.com/rhatiro/previsao-renda/main/ebac-course-utils/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Profissão: Cientista de Dados**
### **Projeto #02** | Previsão de renda

**Aluno:** [Rafael Rosa](https://www.linkedin.com/in/rafael-rosa-alves/)<br>

---

# <div style="text-align:center"> Projeto 02 - Previsão de renda </div>

## Etapa 1 CRISP - DM: Entendimento do negócio

Uma instituição financeira está interessada em aprofundar sua compreensão do perfil de renda de seus novos clientes para várias finalidades, como ajustar de forma mais precisa os limites de crédito dos cartões dos novos clientes, sem a necessidade de solicitar olerites ou documentação que possa afetar a experiência do cliente.

Com esse objetivo, realizou um estudo com alguns clientes, validando suas rendas por meio de olerites e outros documentos, e tem a intenção de desenvolver um modelo preditivo para prever essas rendas com base em algumas variáveis já presentes em seu banco de dados.

## Etapa 2 Crisp-DM: Entendimento dos dados

### Dicionário de dados <a name="dicionario"></a>

| Variável              | Descrição                                                                                                  | Tipo             |
| --------------------- |:----------------------------------------------------------------------------------------------------------:| ----------------:|
| data_ref              | Data de referência de coleta das variáveis                                                                 | object           |
| id_cliente            | Código identificador exclusivo do cliente                                                                  | int              |
| sexo                  | Sexo do cliente (M = 'Masculino'; F = 'Feminino')                                                          | object (binária) |
| posse_de_veiculo      | Indica se o cliente possui veículo (True = 'Possui veículo'; False = 'Não possui veículo')                 | bool (binária)   |
| posse_de_imovel       | Indica se o cliente possui imóvel (True = 'Possui imóvel'; False = 'Não possui imóvel')                    | bool (binária)   |
| qtd_filhos            | Quantidade de filhos do cliente                                                                            | int              |
| tipo_renda            | Tipo de renda do cliente (Empresário, Assalariado, Servidor público, Pensionista, Bolsista)                | object           |
| educacao              | Grau de instrução do cliente (Primário, Secundário, Superior incompleto, Superior completo, Pós graduação) | object           |
| estado_civil          | Estado civil do cliente (Solteiro, União, Casado, Separado, Viúvo)                                         | object           |
| tipo_residencia       | Tipo de residência do cliente (Casa, Governamental, Com os pais, Aluguel, Estúdio, Comunitário)            | object           |
| idade                 | Idade do cliente em anos                                                                                   | int              |
| tempo_emprego         | Tempo no emprego atual                                                                                     | float            |
| qt_pessoas_residencia | Quantidade de pessoas que moram na residência                                                              | float            |
| **renda**             | Valor numérico decimal representando a renda do cliente em reais                                           | float            |

<div style="text-align: center"

### Carregando pacotes necessários

In [ ]:
##pip install ydata-profiling
##pip install ipywidgets

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# from pandas_profiling import ProfileReport
from ydata_profiling import ProfileReport
import os

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree

%matplotlib inline

### Carregando dados necessários

O comando pd.read_csv é um comando da biblioteca pandas (pd.) e carrega os dados do arquivo csv indicado para um objeto *dataframe* do pandas.

In [ ]:
filepath = './input/previsao_de_renda.csv'
renda = pd.read_csv(filepath_or_buffer=filepath)

renda.info()
renda

In [ ]:
renda.nunique()

In [ ]:
renda.drop(columns=['Unnamed: 0', 'id_cliente'], inplace=True)

print('Quantidade total de linhas:', len(renda), '\n')

print('Quantidade de linhas duplicadas:', renda.duplicated().sum(), '\n')

print('Quantidade após remoção das linhas duplicadas:', 
      len(renda.drop_duplicates()), '\n')

renda.drop_duplicates(inplace=True, ignore_index=True)
renda.info()

### Entendimento dos dados - Univariada
Nesta etapa tipicamente avaliamos a distribuição de todas as variáveis. 

#### Pandas Profiling – Relatório interativo para análise exploratória de dados

In [ ]:
prof = ProfileReport(df=renda, 
                     minimal=False, 
                     explorative=True, 
                     dark_mode=True, 
                     orange_mode=True)
os.makedirs(name='./output', exist_ok=True)
prof.to_file('./output/renda_analysis.html')

prof

####  Estatísticas descritivas das variáveis quantitativas

In [ ]:
renda.describe().transpose()

### Entendimento dos dados - Bivariadas

#### Matriz de correlação

In [ ]:
(renda
 .iloc[:,3:]
 .corr()
 .tail(n=1)
)

Com base na matriz de correlação, nota-se que a variável `tempo_emprego` exibe a maior associação com a variável `renda`, apresentando um coeficiente de correlação de 38,5%.

#### Matriz de dispersão 

In [ ]:
sns.pairplot(data=renda, 
             hue='tipo_renda', 
             vars=['qtd_filhos', 
                   'idade', 
                   'tempo_emprego', 
                   'qt_pessoas_residencia', 
                   'renda'], 
             diag_kind='hist')

plt.show()

Ao examinar o *pairplot*, que é uma representação gráfica na forma de matriz de dispersão, é perceptível a presença de alguns *outliers* na variável `renda`, os quais têm o potencial de influenciar os resultados da análise de tendência, embora sejam pouco frequentes. Ademais, nota-se uma correlação fraca entre praticamente todas as variáveis quantitativas, corroborando os achados da matriz de correlação

##### Clustermap

In [ ]:
cmap = sns.diverging_palette(h_neg=100, 
                             h_pos=359, 
                             as_cmap=True, 
                             sep=1, 
                             center = 'light')

ax = sns.clustermap(data=renda.corr(), 
               figsize=(10, 10), 
               center=0, 
               cmap=cmap)
plt.setp(ax.ax_heatmap.get_xticklabels(), rotation=45)

plt.show()

Ao analisar o *pairplot*, uma representação gráfica na forma de matriz de dispersão, é evidente a presença de alguns *outliers* na variável `renda`, os quais possuem o potencial de impactar os resultados da análise de tendência, apesar de sua baixa frequência. Além disso, observa-se uma correlação fraca entre praticamente todas as variáveis quantitativas, o que confirma os resultados obtidos na matriz de correlação.

#####  Linha de tendência

In [ ]:
plt.figure(figsize=(16,9))

sns.scatterplot(x='tempo_emprego',
                y='renda', 
                hue='tipo_renda', 
                size='idade',
                data=renda,
                alpha=0.4)

# Linha de tendência:
sns.regplot(x='tempo_emprego', 
            y='renda', 
            data=renda, 
            scatter=False, 
            color='.3')

plt.show()

Apesar de não ser tão alta, a correlação entre a variável `tempo_emprego` e a variável `renda` revela claramente uma covariância positiva, evidenciada pela inclinação da linha de tendência.

##### Análise de relevância preditiva com variáveis booleanas

In [ ]:
plt.rc('figure', figsize=(12,4))
fig, axes = plt.subplots(nrows=1, ncols=2)

sns.pointplot(x='posse_de_imovel', 
              y='renda',  
              data=renda, 
              dodge=True, 
              ax=axes[0])

sns.pointplot(x='posse_de_veiculo', 
              y='renda', 
              data=renda, 
              dodge=True, 
              ax=axes[1])

plt.show()

Ao contrastar os gráficos acima, é evidente que a variável `posse_de_veículo` é mais relevante na predição de renda, como indicado pela maior disparidade entre os intervalos de confiança para aqueles que possuem e não possuem veículo. Em contrapartida, a variável `posse_de_imóvel` não mostra diferença significativa entre as diferentes condições de posse imobiliária.

In [ ]:
renda['data_ref'] = pd.to_datetime(arg=renda['data_ref'])

qualitativas = renda.select_dtypes(include=['object', 'boolean']).columns

plt.rc('figure', figsize=(16,4))

for col in qualitativas:
    fig, axes = plt.subplots(nrows=1, ncols=2)
    fig.subplots_adjust(wspace=.6)
    
    tick_labels = renda['data_ref'].map(lambda x: x.strftime('%b/%Y')).unique()
    
    # barras empilhadas:
    renda_crosstab = pd.crosstab(index=renda['data_ref'], 
                                 columns=renda[col], 
                                 normalize='index')
    ax0 = renda_crosstab.plot.bar(stacked=True, 
                                  ax=axes[0])
    ax0.set_xticklabels(labels=tick_labels, rotation=45)
    axes[0].legend(bbox_to_anchor=(1, .5), loc=6, title=f"'{col}'")
    
    # perfis médios no tempo: 
    ax1 = sns.pointplot(x='data_ref', y='renda', hue=col, data=renda, dodge=True, ci=95, ax=axes[1])
    ax1.set_xticklabels(labels=tick_labels, rotation=45)
    axes[1].legend(bbox_to_anchor=(1, .5), loc=6, title=f"'{col}'")
    
    plt.show()

---

## Etapa 3 Crisp-DM: Preparação dos dados
Nessa etapa realizamos tipicamente as seguintes operações com os dados:

 - **seleção**: Já temos os dados selecionados adequadamente?
 - **limpeza**: Precisaremos identificar e tratar dados faltantes
 - **construção**: construção de novas variáveis
 - **integração**: Temos apenas uma fonte de dados, não é necessário integração
 - **formatação**: Os dados já se encontram em formatos úteis?

In [ ]:
renda.drop(columns='data_ref', inplace=True)
renda.dropna(inplace=True)

pd.DataFrame(index=renda.nunique().index, 
             data={'tipos_dados': renda.dtypes, 
                   'qtd_valores': renda.notna().sum(), 
                   'qtd_categorias': renda.nunique().values})

### Conversão das variáveis categóricas em variáveis numéricas (dummies)

In [ ]:
renda_dummies = pd.get_dummies(data=renda)
renda_dummies.info()

In [ ]:
(renda_dummies.corr()['renda']
              .sort_values(ascending=False)
              .to_frame()
              .reset_index()
              .rename(columns={'index':'var', 
                               'renda':'corr'})
              .style.bar(color=['darkred', 'darkgreen'], align=0)
)

---

## Etapa 4 Crisp-DM: Modelagem
Nessa etapa que realizaremos a construção do modelo. Os passos típicos são:
- Selecionar a técnica de modelagem
- Desenho do teste
- Avaliação do modelo

Optamos pelo DecisionTreeRegressor como técnica, dada sua aptidão para lidar com problemas de regressão, como a previsão de renda dos clientes. Além disso, as árvores de decisão são de fácil interpretação e possibilitam a identificação dos atributos mais relevantes para a previsão da variável-alvo, o que a torna uma escolha sólida para o projeto.

### Divisão da base em treino e teste

In [ ]:
X = renda_dummies.drop(columns='renda')
y = renda_dummies['renda']

print('Quantidade de linhas e colunas de X:', X.shape)
print('Quantidade de linhas de y:', len(y))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

print('X_train:', X_train.shape)
print('X_test:', X_test.shape)
print('y_train:', y_train.shape)
print('y_test:', y_test.shape)

### Seleção de hiperparâmetros do modelo com for loop

In [ ]:
score = pd.DataFrame(columns=['max_depth', 'min_samples_leaf', 'score'])

for x in range(1, 21):
    for y in range(1, 31):
        reg_tree = DecisionTreeRegressor(random_state=42, 
                                         max_depth=x, 
                                         min_samples_leaf=y)
        reg_tree.fit(X_train, y_train)
        
        score = pd.concat(objs=[score, 
                                pd.DataFrame({'max_depth': [x], 
                                              'min_samples_leaf': [y], 
                                              'score': [reg_tree.score(X=X_test, 
                                                                       y=y_test)]})], 
                          axis=0, 
                          ignore_index=True)
        
score.sort_values(by='score', ascending=False)

### Rodando o modelo

In [ ]:
reg_tree = DecisionTreeRegressor(random_state=42, max_depth=8, min_samples_leaf=4)
reg_tree.fit(X_train, y_train)

#### Visualização gráfica da árvore com plot_tree

In [ ]:
plt.rc('figure', figsize=(18,9))

tp = tree.plot_tree(decision_tree=reg_tree, 
                    feature_names=X.columns, 
                    filled=True)

#### Visualização impressa da árvore

In [ ]:
text_tree_print = tree.export_text(decision_tree=reg_tree)

print(text_tree_print)

## Etapa 5 Crisp-DM: Avaliação dos resultados 

In [ ]:
r2_train = reg_tree.score(X=X_train, y=y_train)
r2_test = reg_tree.score(X=X_test, y=y_test)

template = 'O coeficiente de determinação (𝑅2) da árvore com profundidade = {0} para a base de {1} é: {2:.2f}'

print(template.format(reg_tree.get_depth(), 'treino', r2_train).replace(".", ","))
print(template.format(reg_tree.get_depth(), 'teste', r2_test).replace(".", ","), '\n')

In [ ]:
renda['renda_predict'] = np.round(reg_tree.predict(X), 2)
renda[['renda', 'renda_predict']]

---

## Etapa 6 Crisp-DM: Implantação

Neste cenário foi desenvolvida uma calculadora simples para a simulação das condições de crédito, baseando-se em todo o entendimento que foi feito sobre este modelo ao longo da trajetória da analise

### [Simulando a previsão de renda](https://rhatiro-ebac-projeto02-previsao-renda.streamlit.app/~/+/Simulac%CC%A7a%CC%83o)

In [ ]:
entrada = pd.DataFrame([{'sexo': 'M', 
                         'posse_de_veiculo': False, 
                         'posse_de_imovel': True, 
                         'qtd_filhos': 1, 
                         'tipo_renda': 'Assalariado', 
                         'educacao': 'Superior completo', 
                         'estado_civil': 'Solteiro', 
                         'tipo_residencia': 'Casa', 
                         'idade': 34, 
                         'tempo_emprego': None, 
                         'qt_pessoas_residencia': 1}])
entrada = pd.concat([X, pd.get_dummies(entrada)]).fillna(value=0).tail(1)
print(f"Renda estimada: R${str(np.round(reg_tree.predict(entrada).item(), 2)).replace('.', ',')}")